In [1]:
import time
import numpy as np

np.random.seed(0)

from lib.linear_algebra import Matrix
from lib.nn import Linear, ReLU, NN
from lib.processing import ColumnNormalizer
from lib.metrics.losses import mean_squared_error
from lib.gd_data_loaders import BatchDataLoader, StochasticDataLoader, MiniBatchDataLoader

In [2]:
# Boston House Price Dataset
# The Boston House Price Dataset involves the prediction of a house price in thousands of dollars given details of the house and its neighborhood.
# It is a regression problem. There are 506 observations with 13 input variables and 1 output variable. The variable names are as follows:

data = []
with open("data/boston_house_prices.data", "rt") as f:
    for line in f.readlines():
        data.append([float(v) for v in line.split()])
data = np.array(data)

In [3]:
np.random.shuffle(data)
split = int(len(data) * 0.8)
train_set = data[:split]
test_set = data[split:]

X_train, y_train = train_set[:, :-1], train_set[:, -1:]
X_test, y_test = test_set[:, :-1], test_set[:, -1:]
X_train = Matrix(X_train)
y_train = Matrix(y_train)
X_test = Matrix(X_test)
y_test = Matrix(y_test)
X_train.dims(), y_train.dims(), X_test.dims(), y_test.dims()

((404, 13), (404, 1), (102, 13), (102, 1))

In [4]:
normalizer = ColumnNormalizer()
normalizer.fit(X_train)
X_train = normalizer.transform(X_train)
X_test = normalizer.transform(X_test)
X_train.dims(), X_test.dims()

((404, 13), (102, 13))

In [5]:
nn = NN([
    Linear(13, 4),
    ReLU(),
    Linear(4, 1),
])

In [6]:
time_point = time.time()
for data_loader in [
    BatchDataLoader(X_train, y_train),
    StochasticDataLoader(X_train, y_train),
    MiniBatchDataLoader(X_train, y_train, 32)
]:
    print(data_loader.__class__)
    for i in range(2001):
        X_b, y_b = data_loader.get_batch()
        out = nn(X_b)
        loss = mean_squared_error(y_b, out)
        if i % 100 == 0:
            elapsed_time = int(time.time() - time_point)
            time_point = time.time()
            print(f"{i} | {loss.data:.2f} | {elapsed_time}s") 

        for p in nn.params():
            for v in p.all_values():
                v.zero_grad()
        loss.grad = 1
        loss.backward()

        for p in nn.params():
            for v in p.all_values():
                v.data -= 0.001 * v.grad

    train_out = nn(X_train) 
    train_loss = mean_squared_error(y_train, train_out)
    test_out = nn(X_test) 
    test_loss = mean_squared_error(y_test, test_out)
    print(f"train loss: {train_loss.data:.2f}   test loss: {test_loss.data:.2f}") 

<class 'lib.gd_data_loaders.BatchDataLoader'>
0 | 626.32 | 1s
100 | 47.83 | 223s
200 | 32.07 | 239s
300 | 26.48 | 239s
400 | 23.86 | 251s
500 | 22.20 | 245s
600 | 20.86 | 243s
700 | 19.75 | 234s
800 | 18.88 | 229s
900 | 18.17 | 225s
1000 | 17.59 | 226s
1100 | 17.11 | 231s
1200 | 16.69 | 226s
1300 | 16.34 | 230s
1400 | 16.04 | 224s
1500 | 15.77 | 227s
1600 | 15.57 | 222s
1700 | 15.40 | 228s
1800 | 15.25 | 222s
1900 | 15.10 | 224s
2000 | 14.91 | 230s
train loss: 14.90   test loss: 12.10
<class 'lib.gd_data_loaders.StochasticDataLoader'>
0 | 1.13 | 2s
100 | 19.19 | 0s
200 | 11.66 | 0s
300 | 5.36 | 1s
400 | 0.02 | 0s
500 | 18.38 | 0s
600 | 20.04 | 0s
700 | 149.78 | 0s
800 | 5.25 | 0s
900 | 2.11 | 0s
1000 | 16.03 | 0s
1100 | 2.37 | 0s
1200 | 15.43 | 0s
1300 | 23.04 | 0s
1400 | 101.43 | 0s
1500 | 38.15 | 0s
1600 | 4.92 | 0s
1700 | 5.55 | 0s
1800 | 8.27 | 0s
1900 | 0.55 | 0s
2000 | 15.37 | 0s
train loss: 14.70   test loss: 13.54
<class 'lib.gd_data_loaders.MiniBatchDataLoader'>
0 | 9.43 | 1s


In [8]:
[(round(float(v1),1), float(v2)) for v1, v2 in zip([v[0].data for v in out.values], [v[0].data for v in y_test])][:10]

[(23.3, 20.7),
 (41.0, 39.8),
 (19.4, 17.8),
 (19.6, 19.6),
 (11.5, 14.9),
 (25.0, 22.0),
 (46.4, 48.8),
 (23.0, 25.0),
 (44.4, 48.5),
 (27.2, 23.9)]